In [13]:
import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, StratifiedKFold

from sklearn.pipeline import Pipeline
from sklearn.datasets import make_classification

from scipy.stats import loguniform

from tpot import TPOTClassifier

---

1. С помощью make_classification из sklearn.datasets сгенерируйте датасет с 1 000 объектов, 10 признаками и бинарным таргетом.

In [14]:
# Генерация датасета
X, y = make_classification(
    n_samples=1000,
    n_features=10,
    n_classes=2,
    random_state=42,
)

---

2. Зададим сетку гиперпараметров и инициализируем модель в виде пайплайна. Параметр penalty будем выбирать равновероятно из  ['l1', 'l2'], а параметр регуляризации С из лог-равномерного распределения может принимать значения [10-4, 102].

In [15]:
search_space = {
                'lr__penalty' : ['l1', 'l2'],
                'lr__C' : loguniform.rvs(10**(-4),10**2, size=100)
                }

model = Pipeline([('lr', LogisticRegression(random_state=42, 
                            solver='liblinear'))])

---

Перебор по сетке будем выполнять с помощью GridSearchCV из sklearn.model_selection. 

3. При его инициализации укажем несколько параметров: 

Модель (пайплайн)
Сетку с параметрами
Количество разбиений для (Stratified)KFold, который используется по умолчанию = 3
Метрику для оценки производительности модели с перекрёстной проверкой на тестовом наборе scoring='accuracy'.

4. Обучите GridSearchCV с параметрами (model_grid = grid_search.fit(X, y)).

In [16]:
model_grid = GridSearchCV(
    estimator=model,
    param_grid=search_space,
    cv=StratifiedKFold(3),
    scoring='accuracy'
)

model_grid.fit(X, y)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


GridSearchCV(cv=StratifiedKFold(n_splits=3, random_state=None, shuffle=False),
             estimator=Pipeline(steps=[('lr',
                                        LogisticRegression(random_state=42,
                                                           solver='liblinear'))]),
             param_grid={'lr__C': array([4.06253569e-02, 1.88021470e-03, 2.92165252e-04, 4.80012055e-02,
       1.86053162e+01, 7.88364072e+01, 9.22582182e-02, 5.29708688e-02,
       1.18786929e+01, 2.34938936e-04, 6.59274...
       7.10767706e-03, 3.66392199e+00, 7.53448745e+01, 7.67673081e+00,
       1.36669868e-03, 2.31220631e-01, 1.72905808e-04, 6.52013526e-01,
       1.62262548e-03, 1.44070923e+00, 1.09347392e+00, 7.27938738e-02,
       1.64812861e+01, 1.04941617e-01, 1.84541196e-02, 1.54333062e-03,
       1.89694976e+00, 1.57268070e-02, 8.38609654e-04, 3.67259170e+00,
       4.37924132e+01, 3.23036660e+00, 5.03420969e-03, 3.69680573e-04]),
                         'lr__penalty': ['l1', 'l2']},
             scoring='accuracy')

Теперь можем посмотреть на лучший score и наилучшие гиперпараметры:

In [17]:
print(model_grid.best_score_)
print(model_grid.best_params_)

0.8689947432462403
{'lr__C': 0.002823450760575613, 'lr__penalty': 'l2'}


---

Для случайного поиска воспользуемся RandomizedSearchCV из sklearn.model_selection. Помимо модели, параметров, скоринга и cv, зададим n_iter. Он отвечает за количество выбранных комбинаций параметров. Чем больше n_iter, тем дольше будет работать поиск. Соответственно, максимально возможный n_iter приближает RandomizedSearchCV к GridSearchCV. 

6. Задайте n_iter = 70 и инициализируйте RandomizedSearchCV.

7. Обучите полученный оптимизатор на X и y и оцените .best_score_ и .best_params_.

In [18]:
model_random = RandomizedSearchCV(
    estimator=model,
    param_distributions=search_space,
    cv=StratifiedKFold(3),
    scoring='accuracy',
    n_iter=70
)

model_random.fit(X, y)

print(model_random.best_score_)
print(model_random.best_params_)

0.8689947432462403
{'lr__penalty': 'l2', 'lr__C': 0.002823450760575613}


---

### TPOT

Для генетического алгоритма воспользуемся библиотекой TPOT.

8. Для установки выполните pip install tpot, после чего импортируйте из tpot TPOTClassifier.

9. Задайте параметры немного иначе, при этом оставив те же значения:

In [19]:
search_space = {
                'penalty' : ['l1', 'l2'],
                'C' : loguniform.rvs(10**(-4),10**2, size=100)
                }



10. При инициализации классификатора добавьте следующие параметры:

generations = 5, (Количество поколений в процессе оптимизации)

population_size = 50, (Число особей, сохраняемых в популяции генетического программирования в каждом поколении)

offspring_size = 25, (Количество потомства, которое нужно произвести в каждом поколении генетического программирования)

verbosity = 2, 

config_dict = {'sklearn.linear_model.LogisticRegression': search_space}, (словарь с гиперпараметрами для оптимизации для выбранной модели)

cv = 3, 

scoring = 'accuracy'

In [20]:
# Создание экземпляра TPOTClassifier с заданными параметрами
tpot_classifier = TPOTClassifier(
    generations=5,
    population_size=50,
    offspring_size=25,
    verbosity=2,
    config_dict={"sklearn.linear_model.LogisticRegression": search_space},
    cv=3,
    scoring="accuracy",
)

11. Обучите инициализированный классификатор tpot_classifier.fit(X, y).

In [21]:
tpot_classifier.fit(X,y)

Optimization Progress:   0%|          | 0/175 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: -inf


RuntimeError: There was an error in the TPOT optimization process. This could be because the data was not formatted properly, or because data for a regression problem was provided to the TPOTClassifier object. Please make sure you passed the data to TPOT correctly. If you enabled PyTorch estimators, please check the data requirements in the online documentation: https://epistasislab.github.io/tpot/using/